Colab section

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# !mkdir /content/models
# base_folder = "/content/drive/MyDrive/MRI"
# !cp -a '/content/drive/MyDrive/MRI/dataset' '.'
# !cp -a '/content/drive/MyDrive/MRI/file_list.csv' '.'

In [3]:
# !pip install torchio segmentation-models-pytorch

Training the feature extraction model

In [1]:
feature_extract=False

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch
import torch.nn as nn
import torch.nn.functional as F

class LateFusion3DCNN(nn.Module):
    def __init__(self, num_classes=2):
        super(LateFusion3DCNN, self).__init__()
        # Define separate pathways for each modality with added depth
        self.pathway_mri = nn.Sequential(
            nn.Conv3d(in_channels=1, out_channels=16, kernel_size=3, padding=1),
            nn.BatchNorm3d(16),
            nn.ReLU(),
            nn.MaxPool3d(kernel_size=2, structuctuctide=2),
            nn.Conv3d(in_channels=16, out_channels=32, kernel_size=3, padding=1),
            nn.BatchNorm3d(32),
            nn.ReLU(),
            nn.MaxPool3d(kernel_size=2, stride=2)
        )
        self.pathway_dose = nn.Sequential(
            nn.Conv3d(in_channels=1, out_channels=16, kernel_size=3, padding=1),
            nn.BatchNorm3d(16),
            nn.ReLU(),
            nn.MaxPool3d(kernel_size=2, stride=2),
            nn.Conv3d(in_channels=16, out_channels=32, kernel_size=3, padding=1),
            nn.BatchNorm3d(32),
            nn.ReLU(),
            nn.MaxPool3d(kernel_size=2, stride=2)
        )
        self.pathway_struct = nn.Sequential(
            nn.Conv3d(in_channels=1, out_channels=16, kernel_size=3, padding=1),
            nn.BatchNorm3d(16),
            nn.ReLU(),
            nn.MaxPool3d(kernel_size=2, stride=2),
            nn.Conv3d(in_channels=16, out_channels=32, kernel_size=3, padding=1),
            nn.BatchNorm3d(32),
            nn.ReLU(),
            nn.MaxPool3d(kernel_size=2, stride=2)
        )

        # Adjusted feature extractor to return 128-dimensional vector
        self.feature_extractor = nn.Sequential(
            nn.Flatten(),
            # Adjusted linear layer to output 128 features
            nn.Linear(32 * 3 * 16 * 16 * 16, 128),
            nn.ReLU(),
            nn.Dropout(0.5)
        )

        # Final classification layer, now optional based on feature_extract flag
        self.classifier = nn.Linear(128, num_classes)

    def forward(self, x_mri, x_dose, x_struct, feature_extract=False):
        # Process each modality through its pathway
        x_mri = self.pathway_mri(x_mri)
        x_dose = self.pathway_dose(x_dose)
        x_struct = self.pathway_struct(x_struct)

        # Flatten outputs for concatenation
        x_mri = torch.flatten(x_mri, start_dim=1)
        x_dose = torch.flatten(x_dose, start_dim=1)
        x_struct = torch.flatten(x_struct, start_dim=1)

        # Concatenate the flattened outputs
        x_fused = torch.cat((x_mri, x_dose, x_struct), dim=1)

        # Pass concatenated features through the adjusted feature extractor
        features = self.feature_extractor(x_fused)

        # Return 128-dimensional feature vector if feature_extract is True
        if feature_extract:
            return features

        # Otherwise, classify
        out = self.classifier(features)
        return out



#===============================================================================


import torch
import torch.nn as nn
import torch.nn.functional as F

class MidFusion3DCNN(nn.Module):
    def __init__(self, num_classes=2):
        super(MidFusion3DCNN, self).__init__()
        # Initial separate pathways for MRI, dose, and structural modalities
        self.pathway_mri = nn.Sequential(
            nn.Conv3d(in_channels=1, out_channels=16, kernel_size=3, padding=1),
            nn.BatchNorm3d(16),
            nn.ReLU(),
            nn.MaxPool3d(kernel_size=2, stride=2)
        )
        self.pathway_dose = nn.Sequential(
            nn.Conv3d(in_channels=1, out_channels=16, kernel_size=3, padding=1),
            nn.BatchNorm3d(16),
            nn.ReLU(),
            nn.MaxPool3d(kernel_size=2, stride=2)
        )
        self.pathway_struct = nn.Sequential(
            nn.Conv3d(in_channels=1, out_channels=16, kernel_size=3, padding=1),
            nn.BatchNorm3d(16),
            nn.ReLU(),
            nn.MaxPool3d(kernel_size=2, stride=2)
        )

        # Fusion layer to combine features from all modalities
        self.fusion_conv = nn.Conv3d(in_channels=48, out_channels=64, kernel_size=3, padding=1)
        self.fusion_bn = nn.BatchNorm3d(64)
        self.fusion_pool = nn.MaxPool3d(kernel_size=2, stride=2)

        # Feature extractor part (Adjusted based on the actual output size before flattening)
        self.feature_extractor = nn.Sequential(
            nn.Flatten(),
            # Placeholder for adjusted size; calculate the correct value based on input dimensions
            nn.Linear(64 * 16 * 16 * 16, 128),
            nn.ReLU(),
            nn.Dropout(0.5)
        )

        # Final classification layer
        self.classifier = nn.Linear(128, num_classes)

    def forward(self, x_mri, x_dose, x_struct, feature_extract=feature_extract):
        # Process each modality through its respective pathway
        x_mri = self.pathway_mri(x_mri)
        x_dose = self.pathway_dose(x_dose)
        x_struct = self.pathway_struct(x_struct)

        # Fusion of features from all pathways
        x_fused = torch.cat((x_mri, x_dose, x_struct), dim=1)
        x_fused = F.relu(self.fusion_bn(self.fusion_conv(x_fused)))
        x_fused = self.fusion_pool(x_fused)

        # Extract features
        features = self.feature_extractor(x_fused)

        # If feature extraction mode, return features directly
        if feature_extract:
            return features

        # Otherwise, classify
        out = self.classifier(features)
        return out

#===============================================================================


import torch
import torch.nn as nn
import torch.nn.functional as F

class EarlyFusion3DCNN(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        # Initialize 3D convolution layer
        self.conv1 = nn.Conv3d(in_channels=3, out_channels=16, kernel_size=3, padding=1)
        # Initialize batch normalization for 3D data
        self.bn1 = nn.BatchNorm3d(16)
        # Initialize 3D max pooling layer
        self.pool = nn.MaxPool3d(kernel_size=2, stride=2)

        # Sequential container for feature extraction layers
        self.feature_extractor = nn.Sequential(
            nn.Conv3d(16, 32, 3, padding=1),
            nn.BatchNorm3d(32),
            nn.ReLU(),
            nn.MaxPool3d(2),
            nn.Flatten(),  # Flatten the output for the linear layer
            nn.Linear(32 * 16 * 16 * 16, 128),  # Adjust the input features accordingly
            nn.ReLU(),
            nn.Dropout(0.5)  # Dropout for regularization
        )

        # Classifier with a linear layer
        self.classifier = nn.Linear(128, num_classes)

    def forward(self, x_mri, x_dose, x_struct, feature_extract=feature_extract):
        # Concatenate the input tensors along the channel dimension
        x = torch.cat((x_mri, x_dose, x_struct), dim=1)
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        features = self.feature_extractor(x)

        # Optionally return extracted features before classification
        if feature_extract:
            return features

        # Pass features through the classifier
        out = self.classifier(features)
        return out

# Setup the device (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [3]:
import os
import numpy as np
import pandas as pd
import torch
import torchio as tio
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, WeightedRandomSampler

def load_npy_file(file_path, normalize=True, new_min=0, new_max=1):
    """
    Load a .npy file, optionally apply min-max normalization, and return it.
    """
    try:
        image_data = np.load(file_path).astype(np.float32)  # Ensure data is in float32
        if normalize:
            # Apply min-max normalization
            min_val = image_data.min()
            max_val = image_data.max()
            if max_val - min_val > 0:
                image_data = (image_data - min_val) / (max_val - min_val) * (new_max - new_min) + new_min
            else:
                print(f"Warning: Not normalizing {file_path} due to division by zero.")
        return image_data
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return None

def data_loading_and_augmentation(folder_path='dataset', csv_file_path='file_list.csv'):
    df = pd.read_csv(csv_file_path)
    subjects_list = []

    grouped_df = df.groupby(['pid', 'course', 'lesion'])

    for _, group in grouped_df:
        row = group[['pid', 'course', 'lesion']].iloc[0]
        lesion = f"{row['pid']}_{row['course']}_{row['lesion'].split('.')[0]}"

        group.set_index('modality', inplace=True)
        subject_dict = {'label': group['recurrence'].iloc[0], 'mri': None, 'dose': None, 'struct': None, 'file_names':[], 'lesion':lesion}

        for modality in ['mri', 'dose', 'struct']:
            if modality in group.index:
                file_name = group.loc[modality, 'filename']
                file_path = os.path.join(folder_path, file_name)
                image_data = load_npy_file(file_path)
                # Add a channels dimension
                image_data = image_data[np.newaxis, ...]  # Convert shape (64, 64, 64) to (1, 64, 64, 64)
                # Wrap the numpy array with tio.ScalarImage
                subject_dict[modality] = tio.ScalarImage(tensor=torch.from_numpy(image_data))
                subject_dict['file_names'].append(file_name)

        subject = tio.Subject(
            mri=subject_dict['mri'] if subject_dict['mri'] is not None else tio.ScalarImage(tensor=torch.zeros(1, 1, 64, 64, 64, dtype=torch.float32)),
            dose=subject_dict['dose'] if subject_dict['dose'] is not None else tio.ScalarImage(tensor=torch.zeros(1, 1, 64, 64, 64, dtype=torch.float32)),
            struct=subject_dict['struct'] if subject_dict['struct'] is not None else tio.ScalarImage(tensor=torch.zeros(1, 1, 64, 64, 64, dtype=torch.float32)),
            label=subject_dict['label'],
            lesion=subject_dict['lesion'],
            file_names=subject_dict['file_names']
        )

        subjects_list.append(subject)

    return subjects_list

# Define augmentation transformations
augmentation_transforms = tio.Compose([
    tio.OneOf({
        tio.RandomAffine(scales=0, degrees=180): 1/7,
        tio.RandomAffine(scales=0, degrees=90): 1/7,
        tio.RandomFlip(axes=0): 1/7,
        tio.RandomFlip(axes=1): 1/7,
        tio.RandomAffine(degrees=0, translation=0): 1/7,
        tio.RandomBlur(): 1/7,
        tio.RandomNoise(): 1/7,
    }),
    # tio.transforms.Pad((64, 64, 64)),
])

# Data loading and augmentation
subjects_list = data_loading_and_augmentation()

# Split the data with stratification
train_subjects, temp_subjects, train_labels, temp_labels = train_test_split(data_loading_and_augmentation(), [subject['label'] for subject in subjects_list], test_size=0.3, random_state=42, stratify=[subject['label'] for subject in subjects_list])
val_subjects, test_subjects, val_labels, test_labels = train_test_split(temp_subjects, temp_labels, test_size=0.5, random_state=42, stratify=temp_labels)

In [9]:
for s in val_subjects:
    print(s['lesion'],s['label'])

243_2_Right Motor Cortex 0
420_1_Right Medial Frontal 0
330_1_Right Frontal 0
219_3_Left Medial Cerebellar 0
152_3_Right Median Parietal 0
152_3_Left Frontal Pole 0
152_3_Left Superior Frontal 0
467_1_Left Anterior Temporal 0
243_1_Left Temporal 0
467_1_Left Anterior Frontal 0
338_1_Right Postcentral Gyrus 0
152_3_Left Medial Parietal 0
243_2_Right Superior Cerebellar 1 0
492_1_Left Para Median 0
243_2_Right Frontal 0
152_3_Left Parietal 2 0
492_1_Left Occipital 1 0
257_5_Right Cerebellar 0
243_3_Left Ventricle 1
152_3_Right Anterior Frontal 2 0
492_1_Left Superior Parietal 0
114_1_Left Temporal 0
338_1_Right Superior Frontal Gyrus 1
246_1_Left Frontal 0
152_3_Left Frontal Lateral 0
147_2_Left Medial Occipital 0
463_2_Left Lateral Cerebellum 0
427_1_Left Temporal 0
243_2_Right Occipital 0
246_1_Cerebellar 0
492_2_Right Temporal 0
467_1_Left Insula 1
467_1_Right Occipital 0
257_3_Right Cerebellum 0
492_1_Left Frontal Cavity 0
243_2_Right Superior Medial Occipital 0
427_3_Occipital Tento

In [7]:
def init_weights_he(m):
    if isinstance(m, nn.Conv3d) or isinstance(m, nn.Linear):
        nn.init.kaiming_uniform_(m.weight, mode='fan_in', nonlinearity='relu')
        if m.bias is not None:
            nn.init.constant_(m.bias, 0)


In [8]:
from sklearn.metrics import f1_score
import torch
from torch.cuda.amp import autocast

def evaluate_model_on_dataloader(dataloader, model, device):
    model.eval()  # Set the model to evaluation mode
    y_true, y_pred = [], []

    with torch.no_grad(), autocast(enabled=True):  # Disable gradient calculation for inference
        for subject in dataloader:
            # Extract data and move to the specified device
            mri = subject['mri'][tio.DATA].to(device)
            dose = subject['dose'][tio.DATA].to(device)
            struct = subject['struct'][tio.DATA].to(device)
            labels = subject['label'].to(device)

            # Forward pass: Compute predicted outputs by passing inputs to the model
            outputs = model(mri, dose, struct)
            _, preds = torch.max(outputs, 1)  # Get the predictions

            # Append batch predictions and true labels to lists
            y_pred.extend(preds.cpu().numpy())
            y_true.extend(labels.cpu().numpy())

    # Compute the F1 score between true labels and predictions
    f1 = f1_score(y_true, y_pred, zero_division=0.0)
    return f1


In [9]:
import torch
import torchio as tio
import numpy as np

class FeatureExtractor:
    def __init__(self, model, device):
        self.model = model
        self.device = device

    def extract_features(self, dataloader):
        features = []
        labels = []
        lesions = []
        self.model.to(self.device)
        self.model.eval()  # Ensure the model is in evaluation mode for feature extraction

        with torch.no_grad():  # No gradient needed for feature extraction
            for subject in dataloader:
                # Assuming 'mri', 'dose', 'struct', and 'label' are keys in your dataset
                mri = subject['mri'][tio.DATA].to(self.device)
                dose = subject['dose'][tio.DATA].to(self.device)
                struct = subject['struct'][tio.DATA].to(self.device)
                label = subject['label'].to(self.device)
                lesion = subject['lesion']

                # Extract features
                output_features = self.model(mri, dose, struct, feature_extract=True)

                # Store features and labels
                features.append(output_features.cpu().numpy())  # Convert to numpy array
                labels.append(label.cpu().numpy())
                lesions.append(lesion)

        # Convert lists to numpy arrays for easier handling later
        features = np.concatenate(features, axis=0)
        labels = np.concatenate(labels, axis=0)
        lesions = np.concatenate(lesions, axis=0)
        return features, labels, lesions

    def extract_and_store_features(self, train_dataloader, val_dataloader, test_dataloader):
        train_features, train_labels, train_lesions = self.extract_features(train_dataloader)
        val_features, val_labels, val_lesions = self.extract_features(val_dataloader)
        test_features, test_labels, test_lesions = self.extract_features(test_dataloader)

        # Here you could add additional logic to store these features and labels to disk
        # For simplicity, this function just returns them
        return (train_features, train_labels, train_lesions), (val_features, val_labels, val_lesions), (test_features, test_labels, test_lesions)


In [10]:

#FocalLoss
import torch
import pickle
import numpy as np
import torch.nn as nn
from tqdm import tqdm
import torchio as tio
import seaborn as sns
from torch.optim import AdamW
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch.cuda.amp import GradScaler, autocast
from sklearn.model_selection import StratifiedKFold,KFold
from torch.utils.data import DataLoader, WeightedRandomSampler
from sklearn.metrics import f1_score,precision_score,recall_score,confusion_matrix
from segmentation_models_pytorch.losses import FocalLoss
from transformers import get_linear_schedule_with_warmup

def run_cross_validation(X, model_constructor, learning_rate, batch_size, iteration, n_splits, num_epochs, device):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    output = {"fold": [], "train_losses": [], "val_losses": [], 'best_y_true':  [],'best_y_pred': [],
              'best_f1':  [], 'train_f1': [], 'val_f1': [], 'train_features': [], 'val_features': [],
              'test_features': [], 'best_epoch': []}

    f_train_dataset = tio.SubjectsDataset(train_subjects)
    f_train_dataloader = DataLoader(f_train_dataset, batch_size=batch_size, shuffle=False)
    f_val_dataset = tio.SubjectsDataset(val_subjects)
    f_val_dataloader = DataLoader(f_val_dataset, batch_size=batch_size, shuffle=False)
    f_test_dataset = tio.SubjectsDataset(test_subjects)
    f_test_dataloader = DataLoader(f_test_dataset, batch_size=batch_size, shuffle=False)

    for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
        print(f"Fold {fold + 1}/{n_splits}")
        train_X, val_X = [X[i] for i in train_idx], [X[i] for i in val_idx]

        # Count the class occurrences to determine weights
        class_counts = {0: 0, 1: 0}
        for x in train_X:
            class_counts[x['label']] += 1  # Access the label correctly

        # Calculate weights for each class based on occurrences
        weights_per_class = {cls: 1.0 / count for cls, count in class_counts.items()}

        # Assign a weight to each sample in the training set based on its class
        sample_weights = [weights_per_class[x['label']] for x in train_X]  # Access the label correctly
        sample_weights = torch.tensor(sample_weights, dtype=torch.float)

        # Create a WeightedRandomSampler to handle class imbalance
        sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)

        # Create datasets
        train_dataset = tio.SubjectsDataset(train_X,transform=augmentation_transforms)  # Applies augmentation conditionally
        val_dataset = tio.SubjectsDataset(val_X)  # No augmentation for validation dataset

        # Create dataloaders, utilizing the sampler for the training set to address class imbalance
        train_dataloader = DataLoader(train_dataset, batch_size=batch_size, sampler=sampler, shuffle=False)
        val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

        model = model_constructor().to(device).float()
        model.apply(init_weights_he)

        global model_name # for later ploting
        model_name = model.__class__.__name__

        # Hyperparameters configuration
        num_train_samples = len(train_dataloader.dataset)  # Or compute it manually from your dataset
        num_training_steps = num_train_samples // train_dataloader.batch_size * num_epochs  # Full passes through the dataset for 20 epochs
        num_warmup_steps = round(num_training_steps * 0.1)  # Around 10% of the training data for warmup

        # Instantiate the optimizer
        optimizer = AdamW(model.parameters(), lr=learning_rate)

        # Get the learning rate scheduler
        total_steps = num_training_steps
        warmup_percentage = num_warmup_steps / total_steps
        scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=total_steps)

        # Loss Function
        criterion = FocalLoss('binary', gamma=2.0, alpha=0.1, reduction="mean", ignore_index=-1)


        scaler = GradScaler()
        best_val_loss = float('inf')
        train_losses, val_losses = [], []
        best_f1 = 0.0

        for epoch in range(num_epochs):
            model.train()
            total_train_loss = 0.0
            for subject in tqdm(train_dataloader, desc=f'Epoch {epoch + 1}/{num_epochs} - Training'):
                optimizer.zero_grad()

                mri = subject['mri'][tio.DATA].to(device)
                dose = subject['dose'][tio.DATA].to(device)
                struct = subject['struct'][tio.DATA].to(device)
                labels = subject['label'].to(device)

                # Correct placement of labels_one_hot for FocalLoss
                labels_one_hot = F.one_hot(labels, num_classes=2).float().to(device)

                with autocast():
                    outputs = model(mri, dose, struct,feature_extract=False)
                    loss = criterion(outputs, labels_one_hot)

                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()

                total_train_loss += loss.item()

            avg_train_loss = total_train_loss / len(train_dataloader.dataset)
            train_losses.append(avg_train_loss)

            model.eval()
            total_val_loss = 0.0
            y_pred, y_true = [], [] # it is important to place the initialization here, to we collect only the last ones in the fold
            with torch.no_grad(), autocast():
                for subject in tqdm(val_dataloader, desc=f'Epoch {epoch + 1}/{num_epochs} - Validation'):
                    mri = subject['mri'][tio.DATA].to(device)
                    dose = subject['dose'][tio.DATA].to(device)
                    struct = subject['struct'][tio.DATA].to(device)
                    labels = subject['label'].to(device)
                    labels_one_hot = F.one_hot(labels, num_classes=2).float().to(device)  # Added for validation

                    outputs = model(mri, dose, struct)  # No feature_extract=True
                    loss = criterion(outputs, labels_one_hot)
                    total_val_loss += loss.item()

                    _, preds = torch.max(outputs, 1)
                    y_pred.extend(preds.cpu().numpy())
                    y_true.extend(labels.cpu().numpy())

            avg_val_loss = total_val_loss / len(val_dataloader.dataset)
            val_losses.append(avg_val_loss)

            f1,precision,recall=f1_score(y_true, y_pred,zero_division=0.0),precision_score(y_true, y_pred,zero_division=0.0),recall_score(y_true, y_pred,zero_division=0.0)
            print(f"Epoch {epoch+1}/{num_epochs}: Training Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}")
            print(f"Validation - F1/Precision/Recall: {f1:.4f}/{precision:.4f}/{recall:.4f}")
            print(f"Validation - y_true: {y_true}\nValidation - y_pred: {y_pred}")

            if f1 > best_f1:
                best_f1 = f1
                best_epoch = epoch
                best_y_pred = y_pred
                best_y_true = y_true
                print(f"New best F1 score: {best_f1:.4f}")
                fe = FeatureExtractor(model, device)
                train_features,val_features,test_features = fe.extract_and_store_features(f_train_dataloader, f_val_dataloader, f_test_dataloader)
                train_f1 = evaluate_model_on_dataloader(f_train_dataloader,model,device)
                val_f1 = evaluate_model_on_dataloader(f_val_dataloader,model,device)
            if avg_val_loss < best_val_loss:
                best_val_loss = avg_val_loss
                print(f"New best val loss: {avg_val_loss:.4f}")

            scheduler.step()

        # Accumulate the stats for this iteration and fold
        output['fold'].append(fold)
        output['best_y_true'].append(best_y_true)
        output['best_y_pred'].append(best_y_pred)
        output['train_losses'].append(train_losses)
        output['val_losses'].append(val_losses)
        output['best_f1'].append(best_f1)
        output['train_f1'].append(train_f1)
        output['val_f1'].append(val_f1)
        output['train_features'].append(train_features)
        output['val_features'].append(val_features)
        output['test_features'].append(test_features)
        output['best_epoch'].append(best_epoch)

        # Free up memory (if on GPU)
        del model, optimizer, scheduler, train_dataloader, val_dataloader
        torch.cuda.empty_cache()

    return output


In [11]:
#CrossEntropy
import torch
import pickle
import numpy as np
import torch.nn as nn
from tqdm import tqdm
import torchio as tio
import seaborn as sns
from torch.optim import AdamW
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch.cuda.amp import GradScaler, autocast
from sklearn.model_selection import StratifiedKFold,KFold
from torch.utils.data import DataLoader, WeightedRandomSampler
from sklearn.metrics import f1_score,precision_score,recall_score,confusion_matrix
from transformers import get_linear_schedule_with_warmup

def run_cross_validation(X, model_constructor, learning_rate, batch_size, iteration, n_splits, num_epochs, device):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    output = {"fold": [], "train_losses": [], "val_losses": [], 'best_y_true':  [],'best_y_pred': [],
              'best_f1':  [], 'train_f1': [], 'val_f1': [], 'train_features': [], 'val_features': [],
              'test_features': [], 'best_epoch': []}

    f_train_dataset = tio.SubjectsDataset(train_subjects)
    f_train_dataloader = DataLoader(f_train_dataset, batch_size=batch_size, shuffle=False)
    f_val_dataset = tio.SubjectsDataset(val_subjects)
    f_val_dataloader = DataLoader(f_val_dataset, batch_size=batch_size, shuffle=False)
    f_test_dataset = tio.SubjectsDataset(test_subjects)
    f_test_dataloader = DataLoader(f_test_dataset, batch_size=batch_size, shuffle=False)

    for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
        print(f"Fold {fold + 1}/{n_splits}")
        train_X, val_X = [X[i] for i in train_idx], [X[i] for i in val_idx]

        # Count the class occurrences to determine weights
        class_counts = {0: 0, 1: 0}
        for x in train_X:
            class_counts[x['label']] += 1  # Access the label correctly

        # Calculate weights for each class based on occurrences
        weights_per_class = {cls: 1.0 / count for cls, count in class_counts.items()}

        # Assign a weight to each sample in the training set based on its class
        sample_weights = [weights_per_class[x['label']] for x in train_X]  # Access the label correctly
        sample_weights = torch.tensor(sample_weights, dtype=torch.float)

        # Create a WeightedRandomSampler to handle class imbalance
        sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)

        # Create datasets
        train_dataset = tio.SubjectsDataset(train_X,transform=augmentation_transforms)  # Applies augmentation conditionally
        val_dataset = tio.SubjectsDataset(val_X)  # No augmentation for validation dataset

        # Create dataloaders, utilizing the sampler for the training set to address class imbalance
        train_dataloader = DataLoader(train_dataset, batch_size=batch_size, sampler=sampler, shuffle=False)
        val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

        model = model_constructor().to(device).float()
        model.apply(init_weights_he)

        global model_name # for later ploting
        model_name = model.__class__.__name__

        # Hyperparameters configuration
        num_train_samples = len(train_dataloader.dataset)  # Or compute it manually from your dataset
        num_training_steps = num_train_samples // train_dataloader.batch_size * num_epochs  # Full passes through the dataset for 20 epochs
        num_warmup_steps = round(num_training_steps * 0.1)  # Around 10% of the training data for warmup

        # Instantiate the optimizer
        optimizer = AdamW(model.parameters(), lr=learning_rate)

        # Get the learning rate scheduler
        total_steps = num_training_steps
        warmup_percentage = num_warmup_steps / total_steps
        scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=total_steps)

        # Loss Function
        weights = torch.tensor([1.0, 1.0], dtype=torch.float32).to(device)
        criterion = nn.CrossEntropyLoss(weight=weights)


        scaler = GradScaler()
        best_val_loss = float('inf')
        train_losses, val_losses = [], []
        best_f1 = 0.0

        for epoch in range(num_epochs):
            model.train()
            total_train_loss = 0.0
            for subject in tqdm(train_dataloader, desc=f'Epoch {epoch + 1}/{num_epochs} - Training'):
                optimizer.zero_grad()

                mri = subject['mri'][tio.DATA].to(device)
                dose = subject['dose'][tio.DATA].to(device)
                struct = subject['struct'][tio.DATA].to(device)
                labels = subject['label'].to(device)

                with autocast():
                    outputs = model(mri, dose, struct)
                    loss = criterion(outputs, labels)

                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
                total_train_loss += loss.item()

            avg_train_loss = total_train_loss / len(train_dataloader.dataset)
            train_losses.append(avg_train_loss)

            model.eval()
            total_val_loss = 0.0
            y_pred, y_true = [], [] # it is important to place the initialization here, to we collect only the last ones in the fold
            with torch.no_grad(), autocast():
                for subject in tqdm(val_dataloader, desc=f'Epoch {epoch + 1}/{num_epochs} - Validation'):
                    mri = subject['mri'][tio.DATA].to(device)
                    dose = subject['dose'][tio.DATA].to(device)
                    struct = subject['struct'][tio.DATA].to(device)
                    labels = subject['label'].to(device)

                    outputs = model(mri, dose, struct)  # Again, no feature_extract=True here
                    loss = criterion(outputs, labels)
                    total_val_loss += loss.item()

                    _, preds = torch.max(outputs, 1)
                    y_pred.extend(preds.cpu().numpy())
                    y_true.extend(labels.cpu().numpy())

            avg_val_loss = total_val_loss / len(val_dataloader.dataset)
            val_losses.append(avg_val_loss)

            f1,precision,recall=f1_score(y_true, y_pred,zero_division=0.0),precision_score(y_true, y_pred,zero_division=0.0),recall_score(y_true, y_pred,zero_division=0.0)
            print(f"Epoch {epoch+1}/{num_epochs}: Training Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}")
            print(f"Validation - F1/Precision/Recall: {f1:.4f}/{precision:.4f}/{recall:.4f}")
            print(f"Validation - y_true: {y_true}\nValidation - y_pred: {y_pred}")

            if f1 > best_f1:
                best_f1 = f1
                best_epoch = epoch
                best_y_pred = y_pred
                best_y_true = y_true
                print(f"New best F1 score: {best_f1:.4f}")
                fe = FeatureExtractor(model, device)
                train_features,val_features,test_features = fe.extract_and_store_features(f_train_dataloader, f_val_dataloader, f_test_dataloader)
                train_f1 = evaluate_model_on_dataloader(f_train_dataloader,model,device)
                val_f1 = evaluate_model_on_dataloader(f_val_dataloader,model,device)
            if avg_val_loss < best_val_loss:
                best_val_loss = avg_val_loss
                print(f"New best val loss: {avg_val_loss:.4f}")

            scheduler.step()

        # Accumulate the stats for this iteration and fold
        output['fold'].append(fold)
        output['best_y_true'].append(best_y_true)
        output['best_y_pred'].append(best_y_pred)
        output['train_losses'].append(train_losses)
        output['val_losses'].append(val_losses)
        output['best_f1'].append(best_f1)
        output['train_f1'].append(train_f1)
        output['val_f1'].append(val_f1)
        output['train_features'].append(train_features)
        output['val_features'].append(val_features)
        output['test_features'].append(test_features)
        output['best_epoch'].append(best_epoch)

        # Free up memory (if on GPU)
        del model, optimizer, scheduler, train_dataloader, val_dataloader
        torch.cuda.empty_cache()

    return output


In [12]:
import itertools
import pandas as pd
from datetime import datetime
import pickle

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
df_results=pd.DataFrame()

# Cross-validation setup
num_epochs = 20
model_constructors = [EarlyFusion3DCNN, MidFusion3DCNN, LateFusion3DCNN]
batch_sizes = [4, 8, 16, 32]
learning_rates = [0.01, 0.001, 0.0001]
n_splits = 5
iterations = 1

combination_generator = itertools.product(
    model_constructors,
    [learning_rate for learning_rate in learning_rates],
    [batch_size for batch_size in batch_sizes],
    range(iterations),
    [n_splits],
)

prev_model_constructor = None  # Initialize tracking variable

for config in combination_generator:
    model_constructor, learning_rate, batch_size, iteration, n_splits = config
    print(f"Iteration {iteration+1}/{iterations}")
    model_name = model_constructor().__class__.__name__

    if model_constructor != prev_model_constructor:
        df_results = pd.DataFrame()  # Reset df_results if the model has changed
        prev_model_constructor = model_constructor  # Update the tracking variable

    output = run_cross_validation(train_subjects, model_constructor, learning_rate, batch_size, iteration, n_splits,num_epochs,device)

    model_data = {
        'fold': range(n_splits),
        'model_name': [model_name]*n_splits,
        'loss': ['FocalLoss']*n_splits,
        'num_epochs': [num_epochs]*n_splits,
        'n_splits': [n_splits]*n_splits,
        'batch_size': [batch_size]*n_splits,
        'learning_rate': [learning_rate]*n_splits,
        'iteration': [iteration]*n_splits,
        'iterations': [iterations]*n_splits,
    }

    df1=pd.DataFrame(output)
    df2=pd.DataFrame(model_data)
    df_results = pd.concat([df_results, df1.merge(df2, on='fold')], ignore_index=True)


    # Generate a timestamp
    timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

    # Specify the file path
    file_path = f"{base_folder}/pickle/FeatureExtraction_FocalLoss_{model_name}_{learning_rate}_{batch_size}_{timestamp}.pkl"

    # Saving the dictionary using pickle
    with open(file_path, 'ab') as file:
        pickle.dump(df_results, file)



Iteration 1/1
Fold 1/5


/home/q/.virtualenvs/r/lib/python3.11/site-packages/torch/cuda/amp/grad_scaler.py:126: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
Epoch 1/20 - Training:   0%|          | 0/34 [00:00<?, ?it/s]/home/q/.virtualenvs/r/lib/python3.11/site-packages/torch/amp/autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
Epoch 1/20 - Validation: 100%|██████████| 9/9 [00:02<00:00,  3.07it/s]


Epoch 1/20: Training Loss: 0.2866, Validation Loss: 0.1628
Validation - F1/Precision/Recall: 0.1667/0.1429/0.2000
Validation - y_true: [0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
Validation - y_pred: [1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0]
New best F1 score: 0.1667


/home/q/.virtualenvs/r/lib/python3.11/site-packages/torch/amp/autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
/home/q/.virtualenvs/r/lib/python3.11/site-packages/torch/amp/autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


New best val loss: 0.1628


...

Epoch 3/20 - Training:   0%|          | 0/34 [00:00<?, ?it/s]/home/q/.virtualenvs/r/lib/python3.11/site-packages/torch/amp/autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
Epoch 3/20 - Validation: 100%|██████████| 9/9 [00:03<00:00,  2.54it/s]


Epoch 3/20: Training Loss: 0.9382, Validation Loss: 0.1248
Validation - F1/Precision/Recall: 0.0000/0.0000/0.0000
Validation - y_true: [0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
Validation - y_pred: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
New best val loss: 0.1248


Epoch 4/20 - Training:   0%|          | 0/34 [00:00<?, ?it/s]/home/q/.virtualenvs/r/lib/python3.11/site-packages/torch/amp/autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
Epoch 4/20 - Validation: 100%|██████████| 9/9 [00:03<00:00,  2.38it/s]


Epoch 4/20: Training Loss: 0.2332, Validation Loss: 0.3243
Validation - F1/Precision/Recall: 0.2564/0.1471/1.0000
Validation - y_true: [0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
Validation - y_pred: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
New best F1 score: 0.2564


KeyboardInterrupt: 

In [ ]:
# df_results.to_csv(f"{base_folder}/Last_Two_models_Focal_FeatureExtractionMRI_validation_results.csv",index=False)